In [76]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


df = pd.read_stata('./fwd_holding_merged_10q1-23q2.dta')

df = df[[
    'seriescode',
    'currency',
    'yq',
    'curPur_usd',
    'curSold_usd',
    'netassets'
]]

df['net_position'] = df['curPur_usd'] - df['curSold_usd']
df['percentage to netassets'] = (df['net_position'] / df['netassets']) * 100

In [77]:
# Group by currency and date, keep net only
df_plot = df.groupby(['currency', 'yq'])[['net_position', 'percentage to netassets']].sum().reset_index()
# keep unit to million
df_plot['net_position'] = df_plot['net_position'] / 1e6
display(df_plot[df_plot['currency'] == 'AED'])

,currency,yq,net_position,percentage to netassets
0,AED,2010-07-01,0.000000,0.000000
1,AED,2010-10-01,0.000000,0.000000
2,AED,2013-07-01,0.000000,0.000000
3,AED,2013-10-01,0.000000,0.000000
4,AED,2014-01-01,0.000000,0.000000
5,AED,2014-04-01,0.000000,0.000000
6,AED,2014-07-01,0.000000,0.000000
7,AED,2015-07-01,-89.463708,-12.074119
8,AED,2015-10-01,-100.057883,-14.370038
9,AED,2016-01-01,-100.030637,-16.699398


In [78]:

def currency_plot(df_plot):
    df_plot['bar_q'] = pd.to_datetime(df_plot['yq'], format='%Y%m%d').dt.to_period('Q').astype(str)

    # Initialize the Dash app
    app = dash.Dash(__name__)

    # Define the layout with two sections: one for the bar chart and one for the area chart
    app.layout = html.Div([
        html.H1("Currency Analysis Dashboard", style={'textAlign': 'center'}),

        # Section 1: Horizontal Bar Chart by Date
        html.Div([
            html.H2("Net Position by Date"),
            dcc.Dropdown(
                id='date-dropdown',
                options=[{'label': d, 'value': d} for d in sorted(df_plot['bar_q'].unique())],
                value=sorted(df_plot['bar_q'].unique())[0],  # Default to the first available date
                clearable=False,
                placeholder="Select a date"
            ),
            dcc.Graph(id='bar-chart')
        ], style={'marginBottom': 50, 'width': '90%', 'margin': 'auto'}),

        # Section 2: Area Chart for Multiple Currencies Over Time
        html.Div([
            html.H2("Net Position by Currency Over Time"),
            dcc.Dropdown(
                id='currency-dropdown',
                options=[{'label': cur, 'value': cur} for cur in sorted(df_plot['currency'].unique())],
                value=sorted(df_plot['currency'].unique()),  # Default: all currencies selected
                multi=True,
                placeholder="Select currencies"
            ),
            dcc.Graph(id='area-plot')
        ], style={'width': '90%', 'margin': 'auto'})
    ])

    # Callback to update the horizontal bar chart based on the selected date
    @app.callback(
        Output('bar-chart', 'figure'),
        [Input('date-dropdown', 'value')]
    )
    def update_bar_chart(selected_date):
        filtered_df = df_plot[df_plot['bar_q'] == selected_date]
        fig = px.bar(
            filtered_df,
            x="net_position",
            y="currency",
            orientation="h",
            title=f"Net Position ($ Million) on {selected_date}"
        )
        fig.update_layout(
            xaxis_title="", 
            yaxis_title="Currency",
            template="plotly_white"
        )
        return fig

    # Callback to update the area chart based on the selected currencies
    @app.callback(
        Output('area-plot', 'figure'),
        [Input('currency-dropdown', 'value')]
    )
    def update_area_chart(selected_currencies):
        # If no currency is selected, show an empty DataFrame
        filtered_df = df_plot[df_plot['currency'].isin(selected_currencies)] if selected_currencies else df_plot.iloc[0:0]
        fig = px.area(filtered_df, x="yq", y="net_position", color="currency")
        fig.update_layout(
            xaxis_title="",      # Remove x-axis title
            yaxis_title="Net Position ($ Million)",
            template="plotly_white"
        )
        return fig

    if __name__ == '__main__':
        app.run_server(debug=True)

currency_plot(df_plot)